In [ ]:
!pip install -q peft==0.8.2

In [ ]:
!pip install  datasets==2.14.5

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

c:\Users\AjaykumarKV\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
model_name = "bigscience/bloomz-560m"
# model_name="bigscience/bloom-1b1"
NUM_VIRTUAL_TOKENS = 4
NUM_EPOCHS = 6